In [1]:
midifile = 'data/chopin-fantaisie.mid'

In [2]:
import time
import copy
import subprocess
from abc import abstractmethod

from midipattern import MidiPattern
from distorter import *

# Midi file parser
import midi

device 0 ('ALSA', 'Midi Through Port-0', 0, 1, 0)
device 1 ('ALSA', 'Midi Through Port-0', 1, 0, 0)
device 2 ('ALSA', 'TiMidity port 0', 0, 1, 0)
device 3 ('ALSA', 'TiMidity port 1', 0, 1, 0)
device 4 ('ALSA', 'TiMidity port 2', 0, 1, 0)
device 5 ('ALSA', 'TiMidity port 3', 0, 1, 0)


In [3]:
MidiPattern.MIDI_DEVICE = 2

Init Pygame and Audio
--------

Midi Pattern
--------

In [4]:
pattern = MidiPattern(midi.read_midifile(midifile))
simple = pattern.simplified(bpm=160)
simple.stamp_time('t0')
midi.write_midifile("generated/simple.mid", simple)

In [5]:
print simple.attributes[0][-40:]

[{'t0': 203.24999999999994}, {'t0': 203.25468749999993}, {'t0': 203.34843749999993}, {'t0': 203.34843749999993}, {'t0': 203.43749999999994}, {'t0': 203.44218749999993}, {'t0': 203.53124999999994}, {'t0': 203.53124999999994}, {'t0': 203.62499999999994}, {'t0': 203.62499999999994}, {'t0': 203.71874999999994}, {'t0': 203.71874999999994}, {'t0': 203.81249999999994}, {'t0': 203.81249999999994}, {'t0': 203.90624999999994}, {'t0': 203.90624999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 204.04687499999994}, {'t0': 204.09374999999994}, {'t0': 204.14062499999994}, {'t0': 204.18749999999994}, {'t0': 204.28124999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.54687499999994}, {'t0': 205.59374999999994}, {'t0': 205.64062499999994}, {'t0': 206.99999999999994}, {'t0': 206.99999999

In [6]:
pattern[0]
pattern.play(180)

Was playing note 5 time 2.66666666667


In [7]:
simple.play()

Was playing note 5 time 3.0


Distorter
--------

In [8]:
distorter = VelocityNoiseDistorter(sigma=20.)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-noise.mid', dist_pattern)
dist_pattern.play(bpm=180)

VelocityNoiseDistorter(sigma=8.90)


NameError: global name 'midi' is not defined

In [11]:
print dist_pattern.attributes[0][-4:]

NameError: name 'dist_pattern' is not defined

In [ ]:
distorter = VelocityWalkDistorter(sigma=0.1)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-walk.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [ ]:
distorter = ProgramDistorter()
distorter.randomize()
# for some reason GM 1- 3 makes no sound in pygame?
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/program.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [ ]:
distorter = TempoDistorter(sigma=0, min=0.5, max=2.)
distorter.randomize()
print distorter
print 'time warp', dist_pattern.attributes[0][-4:]
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/tempo.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [ ]:
distorter = TimeNoiseDistorter()
distorter.randomize()
print distorter
print 'time warp', dist_pattern.attributes[0][-4:]
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/time.mid', dist_pattern)
dist_pattern.play(bpm=180)

Individual Note Times to Global Alignment
-------

In [ ]:
def align_frame_to_frame(pattern, stride):
    '''
    Parameters
    ----------
    pattern : MidiPattern
        pattern with alignment attributes
        't0' for reference time
        't' for candidate time (times to align)
    stride : float
        stride of window in seconds
        
    Returns
    -------
    align : list of int
        alignment of each candidate window to index of target window
        
    TODO
    ----
    Smooth using kernel instead of binning candidate + averaging target
    reference by chord/duration of nth note
    interpolate empty reference measures    
        
    Currently, move a non-overlapping window over the pattern.
    Candidate window events have their time averaged.
    It is then assigned to the closest window.
    '''
    # Collect info for each candidate window
    windows = {}
    for track_attributes in pattern.attributes:
        for e_attr in track_attributes:
            ref_idx = int(e_attr['t'] / stride)
            windows.setdefault(ref_idx, [])
            windows[ref_idx].append(e_attr['t0'])
    # Average
    avg = {}
    for cand_idx, w in windows.items():
        avg[cand_idx] = int(sum(t for t in w) / float(len(w)))
    # Fill holes
    align = []
    last = 0.
    for cand_idx, ref_idx in sorted(avg.items()):
        # fill holes with linear interpolation
        repl = np.linspace(float(last), float(ref_idx), num=(cand_idx - len(align) + 2))[1:]
        last = ref_idx
        #repl = [ref_idx] * (cand_idx - len(align) + 1)
        align += list(repl.astype(int))
    return align
        
    

In [ ]:
stride = 1.
align = align_frame_to_frame(dist_pattern, stride)
align

Alignment IO
----

In [ ]:
def write_align(fname, align, stride):
    '''
    Write alignment to file
    '''
    with open(fname, 'w') as f:
        f.write('{}\n'.format(stride))
        f.write('\n'.join(map(str, align)))
        
def read_align(fname):
    '''
    Read alignment from file\
    
    Returns
    -------
    align : list of float
        alignments in seconds for each candidate window
    stride : float
        duration of window
    '''
    with open(fname, 'r') as f:
        numbers = [float(l.strip()) for l in f]
        return numbers[1:], numbers[0]

In [ ]:
write_align('generated/align.txt', align, stride)
align2, stride2 = read_align('generated/align.txt')
print align2 == align
print int(stride2) == int(stride), stride2, stride

In [ ]:
def random_distort(pattern, distorters=None):
    '''
    Distort a simple pattern by applying a chain
    for distortions on it.
    
    Parameters
    ----------
    pattern : MidiPattern
        pattern to distort
    distorters : list of Distorter
        distorters to apply
    '''
    if not distorters:
        distorters = [TempoDistorter(), TimeNoiseDistorter()]
        for distorter in distorters:
            distorter.randomize()
    current = simple
    for i, distorter in enumerate(distorters):
        keep_stamps = i > 0
        current = distorter.distort(current, keep_stamps)
    return current



In [ ]:
dist_pattern = random_distort(simple)
align = align_frame_to_frame(dist_pattern, stride=1.)
print align
dist_pattern.play()

Actual Generation
----

In [ ]:
num_samples = 10
stride = 0.1
for i in xrange(num_samples):
    base_name = 'generated/sample-{}'.format(i)
    align_name = '{}.txt'.format(base_name)
    midi_name = '{}.mid'.format(base_name)
    wav_name = '{}.wav'.format(base_name)
    distorted = random_distort(base_name)
    align = align_frame_to_frame(distorted, stride)
    write_align(align_name, align, stride)
    midi.write_midifile(midi_name, distorted)
    # Convert to wav using timidity
    print wav_name
    subprocess.check_call(['timidity', '-Ow', midi_name, '-o', wav_name])
    print 'Done generating {}'.format(base_name)